In [1]:
import torch

In [38]:
import torch


volume = torch.tensor([[1.0, 1.0], [1.0, 1.0]])[None, None]
coordinates = torch.tensor([-2.0, -2.0])[None, None, None]

values = torch.nn.functional.grid_sample(volume, coordinates, align_corners=True, mode="bilinear", padding_mode="zeros")
print(values)

tensor([[[[0.2500]]]])
